# Threat-aware Swarm — Experiments (launcher notebook)

Этот ноутбук:
- задаёт параметры прогона,
- запускает `scripts/trained_ppo.py`,
- показывает, где лежат логи TensorBoard и модели,
- индексирует/оценивает модели через `scripts/index_models.py` + `scripts/eval_models.py`.

> Все гиперпараметры и версии среды фиксируются в `runs/<run_id>/meta/run.json`, поэтому любой результат можно воспроизвести без ручных правок ячеек.


In [ ]:
import os, sys, subprocess, json
from pathlib import Path

ROOT = Path.cwd()
print("ROOT:", ROOT)

# В Colab обычно нужно перейти в корень репозитория (если он лежит не в текущей папке)
# Пример:
# %cd /content/threat-aware-swarm

# Чтобы импорты вида `from env...` работали в подпроцессах:
os.environ["PYTHONPATH"] = str(ROOT)
print("PYTHONPATH =", os.environ["PYTHONPATH"])


## 1) Параметры эксперимента


In [ ]:
from dataclasses import dataclass

@dataclass
class RunCfg:
    run_name: str = "debug"
    seed: int = 0
    total_timesteps: int = 2_000_000

    # env
    max_steps: int = 600
    goal_radius: float = 3.0

    # vecenv
    num_vec_envs: int = 1
    num_cpus: int = 1
    device: str = "auto"   # auto/cpu/cuda/mps

    # PPO
    learning_rate: float = 3e-4
    n_steps: int = 2048
    batch_size: int = 2048
    n_epochs: int = 10
    gamma: float = 0.99
    gae_lambda: float = 0.95
    ent_coef: float = 0.0
    vf_coef: float = 0.5
    clip_range: float = 0.2
    target_kl: float = 0.01
    net_arch: str = "256,256"
    log_std_init: float = -1.0

    # saving/eval
    eval_freq: int = 200_000
    n_eval_episodes: int = 20
    checkpoint_freq: int = 1_000_000
    deterministic_eval: bool = True

    out_dir: str = "runs"

cfg = RunCfg()
cfg


## 2) Запуск обучения

Команда создаст папку вида `runs/run_YYYYMMDD_HHMMSS/` с подпапками:
- `meta/run.json` — **всё, что нужно для воспроизводимости**
- `tb/` — TensorBoard-логи
- `models/` — checkpoints + best

Если хочешь несколько запусков (sweep) — делай это циклом в отдельной ячейке, но **всё равно** запускай `trained_ppo.py`.


In [ ]:
import shlex, datetime

def run_train(cfg: RunCfg) -> Path:
    cmd = [
        sys.executable, "scripts/trained_ppo.py",
        "--run-name", cfg.run_name,
        "--seed", str(cfg.seed),
        "--total-timesteps", str(cfg.total_timesteps),
        "--max-steps", str(cfg.max_steps),
        "--goal-radius", str(cfg.goal_radius),
        "--device", cfg.device,
        "--num-vec-envs", str(cfg.num_vec_envs),
        "--num-cpus", str(cfg.num_cpus),

        "--learning-rate", str(cfg.learning_rate),
        "--n-steps", str(cfg.n_steps),
        "--batch-size", str(cfg.batch_size),
        "--n-epochs", str(cfg.n_epochs),
        "--gamma", str(cfg.gamma),
        "--gae-lambda", str(cfg.gae_lambda),
        "--ent-coef", str(cfg.ent_coef),
        "--vf-coef", str(cfg.vf_coef),
        "--clip-range", str(cfg.clip_range),
        "--target-kl", str(cfg.target_kl),
        "--net-arch", cfg.net_arch,
        "--log-std-init", str(cfg.log_std_init),

        "--eval-freq", str(cfg.eval_freq),
        "--n-eval-episodes", str(cfg.n_eval_episodes),
        "--checkpoint-freq", str(cfg.checkpoint_freq),
        "--out-dir", cfg.out_dir,
    ]
    if cfg.deterministic_eval:
        cmd.append("--deterministic-eval")

    print("Running:\n", " ".join(map(shlex.quote, cmd)))
    subprocess.run(cmd, check=True, env=os.environ.copy())

    # Ищем последний созданный run_* в out_dir
    out_dir = Path(cfg.out_dir)
    runs = sorted([p for p in out_dir.glob("run_*") if p.is_dir()], key=lambda p: p.stat().st_mtime)
    if not runs:
        raise RuntimeError(f"No run_* directories found in {out_dir}")
    return runs[-1]

run_dir = run_train(cfg)
print("Run dir:", run_dir)
print("Meta:", run_dir / "meta" / "run.json")
print("TB:", run_dir / "tb")
print("Models:", run_dir / "models")


## 3) Быстрая проверка артефактов прогона

Считаем `run.json`, покажем ключевые параметры и наличие моделей.


In [ ]:
run_json = run_dir / "meta" / "run.json"
data = json.loads(run_json.read_text(encoding="utf-8"))
print("Run:", data.get("run_id"), "| created:", data.get("created_at"))
print("Args (subset):")
for k in ["seed","total_timesteps","learning_rate","n_steps","batch_size","gamma","gae_lambda","ent_coef","clip_range","target_kl","net_arch","log_std_init"]:
    if k in data:
        print(f"  {k}: {data[k]}")
print("\nModels:")
models = sorted((run_dir/"models").glob("*.zip"))
for m in models:
    print(" -", m.name, f"({m.stat().st_size/1e6:.1f} MB)")

## 4) TensorBoard

В Colab обычно так:

```python
%load_ext tensorboard
%tensorboard --logdir runs/run_XXXX/tb
```

Локально — `tensorboard --logdir runs/run_XXXX/tb`.


## 5) Индексация и оценка моделей

Одна CSV-таблица на все модели.


In [ ]:
# Индексация конкретно run_dir/models
registry = Path("model_registry.csv")
subprocess.run([sys.executable, "scripts/index_models.py", "--scan", str(run_dir/"models"), "--out", str(registry), "--rewrite"], check=True, env=os.environ.copy())
print("Registry:", registry.resolve())


In [ ]:
# Оценка (если нужно). 
# --mode both: deterministic + stochastic
subprocess.run([sys.executable, "scripts/eval_models.py",
                "--registry", "model_registry.csv",
                "--mode", "both",
                "--n-episodes", "20",
                "--max-steps", str(cfg.max_steps),
                "--deterministic",
                "--seed", "0"], check=True, env=os.environ.copy())
print("Updated registry:", registry.resolve())


## 6) Визуализация лучшей модели

Запусти из терминала (из корня репозитория):

```bash
PYTHONPATH=. python -m viz.run_trained_pz --model runs/run_XXXX/models/best_by_finished.zip
```

или просто подставь нужный путь (можешь копировать его из `run_dir/models`).
